In [2]:
import pandas as pd

In [11]:
df_0_10=pd.read_csv("../extracted_data/data_0_10_1.csv")
df_10=pd.read_csv("../extracted_data/data_10_plus_2.csv")
df_10_2=pd.read_csv("../extracted_data/data_10_plus_1.csv")
df_100=pd.read_csv("../extracted_data/data_100_plus_1.csv")
df_1000=pd.read_csv("../extracted_data/data_1000_plus_1.csv")

In [12]:
df = pd.concat([df_0_10,df_10_2,df_10,df_100,df_1000])

In [13]:
df.shape

(24890, 25)

In [14]:
df.duplicated().sum()

1356

In [15]:
df = df.drop_duplicates()

In [16]:
df.duplicated().sum()

0

In [18]:
df.to_csv('../extracted_data/extracted_data_2.csv', index = False, encoding='utf-8')

In [17]:
df.shape

(23534, 25)